In [ ]:
##Motor + Sensor

In [6]:
from ophyd import Component as Cpt, Device, EpicsSignal, EpicsSignalRO,PVPositioner
from ophyd.status import Status

class MotorDR(PVPositioner):

    setpoint = Cpt(EpicsSignal,':mminput')
    done = Cpt(EpicsSignal, '_Mtr.DMOV') # _Mtr.DMOV
    readback = Cpt(EpicsSignal,':steptomm')



class gsmtr(Device):
    # Señales de setpoint (readback)
    gauss_on = Cpt(EpicsSignal, ":turnon")
    #gauss_off = Cpt(EpicsSignal, ":turnoff")
    #gaus_status_onoff = Cpt(EpicsSignalRO, ":turnon")
    readback = Cpt(EpicsSignalRO, ":getmagfield")
    #done = Cpt(EpicsSignalRO, 'Cmd-Busy')
    def ton(self):
        
        self.gauss_on.put("1")

class pwrspl(Device):
    # Señales de lectura (readback)
    voltage_RB  = Cpt(EpicsSignalRO, ":readV")
    current_RB  = Cpt(EpicsSignalRO, ":readI")
    slew_rate = Cpt(EpicsSignal,":set_slew")
    confirm_slew = Cpt(EpicsSignal,":confirm_slew")

    # Señal de setpoint
    current_SP  = Cpt(EpicsSignal,   ":putI")
    confirm_current = Cpt(EpicsSignal, ":confirm")
    # Señales de encendido/apagado
    on_cmd      = Cpt(EpicsSignal,   ":on")
    off_cmd     = Cpt(EpicsSignal,   ":off")
    # Estado de salida (asumimos que `PWRSPL:on` devuelve 1 cuando está on)
    onoff_status= Cpt(EpicsSignalRO, ":on")

    def set(self, setpoint):
        stat = Status(timeout=30.0)
        print(f"onoff stat: {self.onoff_status.get()}")
        # Asegurar que esté encendido
        if self.onoff_status.get() != "1":
            print("PS is OFF. Switching ON...")
            self.on_cmd.put("1")

        # Poner el setpoint
        self.current_SP.put(setpoint)
        
        #self.confirm_current(0)

        # Esperar hasta que el readback y el setpoint coincidan (±0.1)
        while True:
            if abs(self.current_RB.get() - self.current_SP.get()) < 0.1:
                stat.set_finished()
                break

        return stat

    def toff(self):
        stat = Status(timeout=30.0)
        print(f"onoff stat: {self.onoff_status.get()}")
        # Asegurar que esté encendido
        if self.onoff_status.get() == "1":
            print("PS is ON. Switching OFF...")
            self.on_cmd.put("")

    


In [7]:
motor = MotorDR('GR1:Ax1',name='motor')
gaussmtr = gsmtr('GSMTR',name='gaussmtr')
#gaussmtr.delay=2
psu = pwrspl("PWRSPL", name="psu")

In [ ]:
#motor.readback.read()

## Fixed current and different positions experiment

In [3]:
psu.set(3)

onoff stat: 1


Status(obj=None, done=False, success=False)

In [4]:
from bluesky import RunEngine
from bluesky.plans import scan
from bluesky.callbacks import LiveTable
from ophyd.sim import noisy_det

# Create and configure the RunEngine
RE = RunEngine()
#print("%f",mot.motor_rdist_mm)
gaussmtr.ton()
psu.set(6)
motor.settle_time = 2
# Subscribe a LiveTable to show the current readback
RE.subscribe(LiveTable([motor.readback,psu.current_RB,gaussmtr.readback]))

# Perform a scan moving the PSU setpoint from 0 A to 7 A in 11 steps
RE(scan([psu.current_RB,gaussmtr.readback], motor, 0,20, 21))

onoff stat: 1


+-----------+------------+----------------+----------------+-------------------+
|   seq_num |       time | motor_readback | psu_current_RB | gaussmtr_readback |
+-----------+------------+----------------+----------------+-------------------+
|         1 | 13:51:51.3 |             -0 |              6 |               199 |
|         2 | 13:51:53.7 |              1 |              6 |               199 |
|         3 | 13:51:56.1 |              2 |              6 |               201 |
|         4 | 13:51:58.5 |              3 |              6 |               201 |
|         5 | 13:52:00.9 |              4 |              6 |               202 |
|         6 | 13:52:03.3 |              5 |              6 |               202 |
|         7 | 13:52:05.7 |              6 |              6 |               203 |
|         8 | 13:52:08.1 |              7 |              6 |               202 |
|         9 | 13:52:10.5 |              8 |              6 |               203 |
|        10 

CA client library is unable to contact CA repeater after 50 tries.
Silence this message by starting a CA repeater daemon
or by calling ca_pend_event() and or ca_poll() more often.


|        18 | 13:52:32.1 |             17 |              6 |               204 |
|        19 | 13:52:34.5 |             18 |              6 |               205 |
|        20 | 13:52:36.9 |             19 |              6 |               205 |
|        21 | 13:52:39.3 |             20 |              6 |               205 |
+-----------+------------+----------------+----------------+-------------------+
generator scan ['b45c59c5'] (scan num: 1)




('b45c59c5-b024-4b42-89d7-bee996067fe2',)

## Fixed position and different currents experiment

In [10]:
from bluesky import RunEngine
from bluesky.plans import scan
from bluesky.callbacks import LiveTable
from ophyd.sim import noisy_det

# Create and configure the RunEngine
RE = RunEngine()
#print("%f",mot.motor_rdist_mm)
gaussmtr.ton()
psu.slew_rate.put(12)
psu.confirm_slew.put(1)
#psu.set(0)

# Subscribe a LiveTable to show the current readback
# RE.subscribe(LiveTable([psu.current_RB,motor.readback,gaussmtr.readback]))
RE.subscribe(LiveTable([motor.readback,psu.current_RB,gaussmtr.readback]))

# Perform a scan moving the PSU setpoint from -6 A to 6 A in 11 steps
RE(scan([motor.readback, gaussmtr.readback], psu, -6,6, 13))

onoff stat: 1


+-----------+------------+----------------+----------------+-------------------+
|   seq_num |       time | motor_readback | psu_current_RB | gaussmtr_readback |
+-----------+------------+----------------+----------------+-------------------+
|         1 | 14:02:12.0 |             20 |             -6 |              -422 |
onoff stat: 1
|         2 | 14:02:13.0 |             20 |             -5 |              -458 |
onoff stat: 1
|         3 | 14:02:14.0 |             20 |             -4 |              -398 |
onoff stat: 1
|         4 | 14:02:15.0 |             20 |             -3 |              -338 |
onoff stat: 1
|         5 | 14:02:16.0 |             20 |             -2 |              -278 |
onoff stat: 1
|         6 | 14:02:17.0 |             20 |             -1 |              -215 |
onoff stat: 1
|         7 | 14:02:18.0 |             20 |              0 |              -152 |
onoff stat: 1
|         8 | 14:02:19.0 |             20 |              1 |               -

('4ebcf72d-5f72-4a36-8fe7-3cc1107b9e85',)

CA.Client.Exception...............................................
    Context: "iocsc3ess4x.ews.bessy.de:45627"
    Source File: ../cac.cpp line 1237
    Current Time: Thu Aug 28 2025 14:04:43.339849317
..................................................................


In [ ]:
RE.halt()